# Library and Root setup

In [1]:
import os
import glob
import pandas as pd

#SQL Folder root
rootdir = "C:\\Users\\Admin\\OneDrive - Bytedance\\R1 APAC - Quality\\Raw Data\\Productivity"


#Standard files root
teamcode_rootdir = 'C:\\Users\\Admin\\OneDrive - Bytedance\\R1 APAC - Quality\\PowerBI\\Team code.xlsx'
queues_rootdir = 'C:\\Users\\Admin\\OneDrive - Bytedance\\R1 APAC - Quality\\PowerBI\\Queues - Raw Data.xlsx'


#Extraction files root
sql_rootdir = 'C:\\Users\\Admin\\OneDrive - Bytedance\\R1 APAC - Quality\\Processed Data\\SQL - Raw Data.xlsx'


# Combining files and Spotcheck

In [2]:
### Extracting file paths
all_filenames = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        all_filenames.append(os.path.join(subdir, file))

### Creating empty Dataframes
combined_xlsx = pd.DataFrame() #Empty Dataframe
combined_csv = pd.DataFrame() #Empty Dataframe 

### Looping through files
for file in all_filenames:

    ### xlsx files
    if file.endswith('.xlsx'):
        exce_file= pd.concat(pd.read_excel(file ,sheet_name=None), ignore_index = True)
        
        ### Controlling Empty files
        if exce_file.empty:
            print(file, '   --- File is empty')
            continue
        
        ### Controlling SQL file structure
        elif 'QA Total Handling Time (s)' not in exce_file.columns:
            print(file, "   --- Not a SQL file")
            continue
            
        ### Combining xlsx files
        elif "QA Total Handling Time (s)" in exce_file.columns: 
            combined_xlsx = combined_xlsx.append(exce_file, ignore_index=True)

    ### csv files
    elif file.endswith('.csv'):
        csv_file= pd.read_csv(file, encoding= 'unicode_escape')
        
        ### Controlling Empty files       
        if csv_file.empty:
            print(file, '   --- File is empty')
            continue
            
        ### Controlling SQL file structure. 
        elif "QA Total Handling Time (s)" not in csv_file:
            print(file, "   --- Not a SQL file")
            continue
            
        ### Combining csv files
        elif "QA Total Handling Time (s)" in csv_file.columns: 
            combined_csv = combined_csv.append(csv_file, ignore_index=True)

    else:
        ### Controlling file extension.
        print(file, '   ---Not an Excel file')

### Combining dataframes
combined_all = pd.concat([combined_xlsx, combined_csv], ignore_index=True)
combined_all = combined_all.reset_index()
print("\n")
combined_all.info()

C:\Users\Admin\OneDrive - Bytedance\R1 APAC - Quality\Raw Data\Productivity\Test files\Claimant file.xlsx    --- Not a SQL file
C:\Users\Admin\OneDrive - Bytedance\R1 APAC - Quality\Raw Data\Productivity\Test files\Empty file test.xlsx    --- File is empty


C:\Users\Admin\AppData\Local\Temp\ipykernel_10456\3755396959.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_xlsx = combined_xlsx.append(exce_file, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10456\3755396959.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_xlsx = combined_xlsx.append(exce_file, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10456\3755396959.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_xlsx = combined_xlsx.append(exce_file, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10456\3755396959.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083 entries, 0 to 1082
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       1083 non-null   int64  
 1   Date                        1083 non-null   object 
 2   QA Name                     1083 non-null   object 
 3   Sampling Queue ID           1083 non-null   int64  
 4   Sampling Queue Title        1083 non-null   object 
 5   QA Total Handling Time (s)  1083 non-null   float64
 6   No. of QA Tasks             1083 non-null   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 59.4+ KB


C:\Users\Admin\AppData\Local\Temp\ipykernel_10456\3755396959.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_xlsx = combined_xlsx.append(exce_file, ignore_index=True)


# Renaming Columns

In [3]:
df = combined_all.rename(columns = {'No. of QA Tasks': 'object_id'
                               ,'QA Name': 'QA_Name'
                               ,'QA Total Handling Time (s)': 'Handling_Time'
                               ,'Sampling Queue Title':'Queue_Name'
                               ,'Sampling Queue ID':'project_id'
                              }).copy()

# Calculating and Adding WeekEnding column

In [4]:
# I must add the data range as August Week 3 otherwise it will update with the first value that meets the conditions. 
ls = []
for i in df['Date']:
    d = str(i).split('-')
        
      #2022
    if int(d[0])== 2022:
        
         #January
        if int(d[1]) == 1:
            if (int(d[2]) >= 1 and int(d[2]) <= 2):  #Jan week 1
                ls.append('2022-01-02')
            elif (int(d[2]) >= 3 and int(d[2]) <= 9):  #Jan week 2
                ls.append('2022-01-09')
            elif  (int(d[2]) >= 10 and int(d[2]) <= 16):  #Jan week 3
                ls.append('2022-01-16')
            elif (int(d[2]) >= 17 and int(d[2]) <= 23):  #Jan week 4
                ls.append('2022-01-23')
            elif (int(d[2]) >= 24 and int(d[2]) <= 30):  #Jan week 4
                ls.append('2022-01-30')
            elif (int(d[2]) == 31):  #Jan week 4
                ls.append('2022-02-06')
        
        #February
        elif int(d[1]) == 2: 
            
            if (int(d[2]) >= 1 and int(d[2]) <= 6):  #Feb week 1
                ls.append('2022-02-06')
            elif (int(d[2]) >= 7 and int(d[2]) <= 13):  #Feb week 2
                ls.append('2022-02-13')
            elif (int(d[2]) >= 14 and int(d[2]) <= 20):  #Feb week 3
                ls.append('2022-02-20')
            elif (int(d[2]) >= 21 and int(d[2]) <= 27):  #Feb week 4
                ls.append('2022-02-27')
            elif (int(d[2]) == 28):  #Feb week 4
                ls.append('2022-03-06')
        
        #March
        elif int(d[1]) == 3:    
            if (int(d[2]) >= 1 and int(d[2]) <= 6):  #March week 1
                ls.append('2022-03-06')
            elif (int(d[2]) >= 7 and int(d[2]) <= 13):  #March week 2
                ls.append('2022-03-13')
            elif (int(d[2]) >= 14 and int(d[2]) <= 20):  #March week 3
                ls.append('2022-03-20')
            elif (int(d[2]) >= 21 and int(d[2]) <= 27):  #March week 4
                ls.append('2022-03-27')
            elif (int(d[2]) >= 28 and int(d[2]) <= 31):  #March week 4
                ls.append('2022-04-03')
            
        #April
        elif int(d[1]) == 4:    
            if (int(d[2]) >= 1 and int(d[2]) <= 3):  #April week 1
                ls.append('2022-04-03')
            elif (int(d[2]) >= 4 and int(d[2]) <= 10): #April week 2
                ls.append('2022-04-10')
            elif (int(d[2]) >= 11 and int(d[2]) <= 17): #April week 3
                ls.append('2022-04-17')
            elif (int(d[2]) >= 18 and int(d[2]) <= 24):  #April week 4
                ls.append('2022-04-24')
            elif (int(d[2]) >= 25 and int(d[2]) <= 30):  #April week 5
                ls.append('2022-05-01')
                
        #May
        elif int(d[1]) == 5:    
            if (int(d[2]) >= 1 and int(d[2]) <= 1):  #May week 1
                ls.append('2022-05-01')
            elif (int(d[2]) >= 2 and int(d[2]) <=8): #May week 2
                ls.append('2022-05-08')
            elif (int(d[2]) >= 9 and int(d[2]) <= 15): #May week 3
                ls.append('2022-05-15')
            elif (int(d[2]) >= 16 and int(d[2]) <= 22):  #May week 4
                ls.append('2022-05-22')
            elif (int(d[2]) >= 23 and int(d[2]) <= 29):  #May week 5
                ls.append('2022-05-29')
            elif (int(d[2]) >= 30 and int(d[2]) <= 31):  #May week 5
                ls.append('2022-06-05')
        #June
        elif int(d[1]) == 6:    
            if (int(d[2]) >= 1 and int(d[2]) <= 5):  #June week 1
                ls.append('2022-06-05')
            elif (int(d[2]) >= 6 and int(d[2]) <=12): #June week 2
                ls.append('2022-06-12')
            elif (int(d[2]) >= 13 and int(d[2]) <= 19): #June week 3
                ls.append('2022-06-19')
            elif (int(d[2]) >= 20 and int(d[2]) <= 26):  #June week 4
                ls.append('2022-06-26')
            elif (int(d[2]) >= 27 and int(d[2]) <= 30):  #June week 5
                ls.append('2022-07-03')
                
        #July
        elif int(d[1]) == 7:    
            if (int(d[2]) >= 1 and int(d[2]) <= 3):  #July week 1
                ls.append('2022-07-03')
            elif (int(d[2]) >= 4 and int(d[2]) <=10): #July week 2
                ls.append('2022-07-10')
            elif (int(d[2]) >= 11 and int(d[2]) <= 17): #July week 3
                ls.append('2022-07-17')
            elif (int(d[2]) >= 18 and int(d[2]) <= 24):  #July week 4
                ls.append('2022-07-24')
            elif (int(d[2]) >= 27 and int(d[2]) <= 31):  #July week 5
                ls.append('2022-07-31')
        
        else:
            ls.append('Wrong')        
            
df['WeekEnding']= pd.DataFrame(ls) #Adding Week ending to the dataset.

# Reordering the columns

In [5]:
df = df[['WeekEnding','Date', 'QA_Name', 'Queue_Name', 'object_id', 'Handling_Time']].copy()

# Creating new columns (TL, Code, Market)

In [6]:
#Path and doc with team codes
df_tc = pd.read_excel(teamcode_rootdir)
df = df.join(df_tc.set_index('QA')['TL'], on='QA_Name') # Adding TL
df = df.join(df_tc.set_index('QA')['Code'], on='QA_Name') # Adding Code
df = df.join(df_tc.set_index('QA')['Market'], on='QA_Name') # Adding Market
df.fillna('Unmapped', inplace = True) # Controling null values due to lack of data

# Adding Column BPO and Type

In [7]:
#Path and doc with team codes
df_qrd = pd.read_excel(queues_rootdir)
df_qrd.info()
df = df.join(df_qrd.set_index('Queue Name')['BPO'], on='Queue_Name') # Adding BPO
df = df.join(df_qrd.set_index('Queue Name')['Type'], on='Queue_Name') # Adding Queue type
df.fillna('Unmapped', inplace = True) # Controling null values due to lack of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7663 entries, 0 to 7662
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Queue Name  7663 non-null   object
 1   Queue ID    7663 non-null   int64 
 2   Type        7663 non-null   object
 3   BPO         4450 non-null   object
dtypes: int64(1), object(3)
memory usage: 239.6+ KB


# Reshaping the dataframe

In [8]:
df = df.rename(columns={'Date': 'date'
                        , 'QA_Name': 'verifier'
                        , 'Handling_Time': 'Handling_Time_Seconds'
                        , 'Queue_Name': 'Queue Name'})
df["timestamp"] = ""
df["hrt"] = ""
df["project_id"] = ""
df['WeekEnding'] = pd.to_datetime(df['WeekEnding'])

# Exporting to excel file

In [9]:
with pd.ExcelWriter(sql_rootdir,
                        date_format='dd/mm/yyy',
                        datetime_format='dd/mm/yyyy') as writer:
    df.to_excel(writer, index=False, encoding='utf-8-sig', sheet_name= 'SQL - Raw Data')  

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083 entries, 0 to 1082
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   WeekEnding             1083 non-null   datetime64[ns]
 1   date                   1083 non-null   object        
 2   verifier               1083 non-null   object        
 3   Queue Name             1083 non-null   object        
 4   object_id              1083 non-null   int64         
 5   Handling_Time_Seconds  1083 non-null   float64       
 6   TL                     1083 non-null   object        
 7   Code                   1083 non-null   object        
 8   Market                 1083 non-null   object        
 9   BPO                    1083 non-null   object        
 10  Type                   1083 non-null   object        
 11  timestamp              1083 non-null   object        
 12  hrt                    1083 non-null   object        
 13  pro